# CHAIN

#### a `chain` refers to a sequence of operations or steps that are linked together to perform a specific task. 

Chains allows to :
 combine mutiple components together to create a single, coherent applicatoin.
    e.g.: a chain can takes user inputs, formats it with a PromptTemplate, and then passes the formatted response to an LLM.

In [1]:
import sys
import os
from dotenv import load_dotenv, find_dotenv

# Import "connections" (connections.py) from folder c02_llm_models
sys.path.append(os.path.join(os.getcwd(), '..'))
from c02_llm_model_connection.connections import create_llm_chat_langchain, create_llm_langchain

load_dotenv(find_dotenv())

AZURE_AOAI_API_VERSION = "2024-08-01-preview"
AZURE_AOAI_MODEL_GPT3_TURBO = "gpt35turbo"
AZURE_AOAI_MODEL_GPT4O = "gpt4o"
AZURE_AOAI_MODEL_GPT4OMINI = "gpt4omini"
AZURE_EMBEDDING_MODEL = "text-embedding-ada"

In [2]:
# call chat model  
llm_chat = create_llm_chat_langchain(AZURE_AOAI_MODEL_GPT4OMINI, AZURE_AOAI_API_VERSION)

TypeError: AsyncClient.__init__() got an unexpected keyword argument 'proxies'

### 1 Old way to create a chain - LEGACY
##### 1.1 Basic Single chain

In [10]:
from langchain_core.prompts import ChatPromptTemplate,HumanMessagePromptTemplate

humain_message_template = HumanMessagePromptTemplate.from_template(
    "Make up a funny comapny for a company that produces {product}"
)

chat_prompt_template = ChatPromptTemplate.from_messages([humain_message_template])


In [11]:
from langchain.chains import LLMChain
old_chain = LLMChain(llm = llm_chat, prompt= chat_prompt_template)


In [12]:
# print(old_chain.invoke("Computers")) # this works also as it is a single input
# print(old_chain.invoke({"Computers"})) # better way is to include in a dictionary foramt
print(old_chain.invoke(input={"Computers"})) # best way is to declare "input" variable

{'product': {'Computers'}, 'text': '**Company Name:** ByteMe Computers\n\n**Tagline:** "We take a byte out of your tech troubles!"\n\n**About Us:** At ByteMe Computers, we believe that technology should be as fun as it is functional. Our mission is to create computers that not only perform like a dream but also bring a smile to your face. Whether you\'re gaming, working, or just browsing cat videos, our computers are designed to handle it all—while making you chuckle along the way!\n\n**Product Highlights:**\n- **The GiggleBox:** A computer that tells jokes while you work. Need a break? Just ask it for a pun!\n- **The LOLaptop:** Lightweight and portable, this laptop comes with a built-in laugh track to keep you entertained during those long meetings.\n- **The ChuckleStation:** A gaming rig that rewards you with funny sound effects every time you level up or achieve a new high score.\n\n**Customer Support:** Our tech support team is available 24/7 and is trained in both troubleshooting

##### 1.2 -  Simple Sequential Chain

The simplest form of sequential chains, where each step has a singular input/output, and the output of one step is the input to the next.
Executes a linear sequence of chains.

In [15]:
from langchain.chains import SimpleSequentialChain
from langchain_core.prompts import PromptTemplate

# rapper
template1: str = """You are an American rapper, your job is to come up with\
lyrics based on a given topic

Here is the topic you have been asked to generate a lyrics on:
{input}\
"""

prompt1: PromptTemplate = PromptTemplate(
    input_variables=["input"], template=template1)


# verifier
template2: str = """You are a verifier of rap songs, you are tasked\
to inspect the lyrics of rap songs. If they consist of violence and abusive languge\
flag the lyrics. Your response should be only one word either True or False.

Here is the lyrics submitted to you:
{input}\
"""
prompt2: PromptTemplate = PromptTemplate(
    input_variables=["input"], template=template2)

chain1 = LLMChain(llm=llm_chat, prompt=prompt1)
chain2 = LLMChain(llm=llm_chat, prompt=prompt2)

`verbose = True` provides detailed information about each step of the process, which is useful for debugging and understanding the data flow.

In [16]:
ss_chain: SimpleSequentialChain = SimpleSequentialChain(
    chains=[chain1, chain2], verbose = True)

# running chain
review: str = ss_chain.invoke("generative ai topic")
print(review)



> Entering new SimpleSequentialChain chain...
**Verse 1**  
Yo, I’m spittin’ rhymes with a digital twist,  
Generative AI, can’t resist,  
Lines crafted by code, it’s a lyrical feat,  
Machine learning magic, feel the beat.  

From text to art, it’s a brand new game,  
Creativity unleashed, ain’t it insane?  
Neural networks flexin’, they’re breakin’ the mold,  
Stories and visions, watch ‘em unfold.  

**Chorus**  
Generative AI, we’re risin’ high,  
Creating the future, reachin’ for the sky,  
From beats to bars, it’s a brand new vibe,  
In this digital world, we’re alive, we thrive.  

**Verse 2**  
Got algorithms spinnin’, they’re takin’ the stage,  
Craftin’ new worlds, turnin’ the page,  
From poetry to portraits, it’s all in the flow,  
AI’s the artist, watch the magic grow.  

Collabs with the code, it’s a fusion of minds,  
Human and machine, see what we find,  
Inspiration’s a spark, ignitin’ the fire,  
Together we rise, takin’ it higher.  

**Chorus**  
Generative AI, we’

##### 1.2 - Sequential Chain
Also executes a sequence of chains, but with added flexbility in passing and tracking outputs.

In [ ]:
template1 = "Give a summayr of this employee's performance review:\n {review}"
prompt1 = ChatPromptTemplate.from_template(template1)
chain1 = LLMChain(llm=llm_chat, prompt=prompt1, output_key="review_summary")

template2 = "Identify key employee weaknesses in this review summary:\n {review_summary}"
prompt2 = ChatPromptTemplate.from_template(template1)
chain2 = LLMChain(llm=llm_chat, prompt=prompt2, output_key="weaknesses")

template3 = "Create:\n {review_summary}"
prompt3 = ChatPromptTemplate.from_template(template1)
chain3 = LLMChain(llm=llm_chat, prompt=prompt3, output_key="weaknesses")

### 2 New way to create a chain (LangChain Expression Language (LCEL) )

`LCEL :  prompt | llm | ... | ... `


In [17]:

humain_message_template = HumanMessagePromptTemplate.from_template(
    "Make up a funny comapny for a company that produces {product}"
)

chat_prompt_template = ChatPromptTemplate.from_messages([humain_message_template])

new_chain_LCEL = chat_prompt_template | llm_chat

result = new_chain_LCEL.invoke(input={"Computers"})
print(result) 
print('-'*100)
print(result.content)

content='**Company Name:** ByteMe Computers\n\n**Tagline:** "We put the \'fun\' in functional!"\n\n**About Us:** At ByteMe Computers, we believe that technology should be as entertaining as it is efficient. Our computers come pre-loaded with quirky wallpapers, dad jokes, and a built-in "procrastination mode" that plays soothing nature sounds while you "work." \n\n**Product Highlights:**\n- **The SnoreBook:** Perfect for those long meetings—this laptop features a built-in pillow and a snooze button that activates a 10-minute power nap.\n- **The GiggleBox:** A desktop that tells jokes every time you boot it up. Warning: May cause spontaneous laughter in the office.\n- **The SnackTop:** Equipped with a mini-fridge and a snack dispenser, because who says you can’t munch while you crunch numbers?\n\n**Mission:** To make computing a delightful experience, one byte at a time!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 190, 'prompt_tokens': 23,

##### 2.1 - Sequential Chaining with LCEL

In [19]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough    # for chaining 
from langchain_core.prompts import PromptTemplate
from langchain.callbacks.tracers import ConsoleCallbackHandler  # callback handler to the "invoke" method's configuration; SIMILAR to VERBOSE = TRUE

context = "I run a blog; I address a semi-technical audience on LinkedIn; my writing style uses commonly understandable language to make it sound like a personal blog post using anecdotes and jokes while keeping an overall professional tone"
goal = "be as creative as possible; take my initial ideas and augment, optimize given the context above; function as my creative writing assistant"
topic = "Data & AI"
thoughts = "Data & AI related stuff"

structure_prompt = PromptTemplate.from_template(
    """You are a writer. Given the following insights you are tasked with brainstorming a structure for a new blog post; be creative, add things I might have not considered and build a structure for an informative yet engaging blog post
    Context: {context}
    Goal: {goal}
    Topic: {topic}
    Thoughts: {thoughts}
    """
)

review_prompt = PromptTemplate.from_template(
    """You are an expert reviewer. Given this drafted structure for a blog post, what would you optimize; what would you add, focus or remove given the context, topic and thoughts explained above:
    Structure: {structure}
    Provide concrete feedback in a cohesive and comprehensive form that a writer can optimize the original sturcture accordingly.
    """
)

optimization_prompt = PromptTemplate.from_template(
    """You are a writer. Optimize the following blog post structure given the feedback review provided.
    Structure: {structure}
    Review: {review}
    """
)


structure_chain = structure_prompt | llm_chat | StrOutputParser()
review_chain = review_prompt | llm_chat | StrOutputParser()
optimization_chain = optimization_prompt | llm_chat | StrOutputParser()



chain = ({"structure" : structure_chain} 
        | RunnablePassthrough.assign(review=review_chain)
        | RunnablePassthrough.assign(optimization=optimization_chain))

result = chain.invoke({"context": context, "goal" : goal, "topic" : topic, "thoughts" : thoughts}, config={'callbacks': [ConsoleCallbackHandler()]})

print(result)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "context": "I run a blog; I address a semi-technical audience on LinkedIn; my writing style uses commonly understandable language to make it sound like a personal blog post using anecdotes and jokes while keeping an overall professional tone",
  "goal": "be as creative as possible; take my initial ideas and augment, optimize given the context above; function as my creative writing assistant",
  "topic": "Data & AI",
  "thoughts": "Data & AI related stuff"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<structure>] Entering Chain run with input:
{
  "context": "I run a blog; I address a semi-technical audience on LinkedIn; my writing style uses commonly understandable language to make it sound like a personal blog post using anecdotes and jokes while keeping an overall professional tone",
  "goal": "be as creative as possible; take my initial ideas and augment, optimize given the context above; fu

Other chain types to test : 

RouterChain: This chain is used to route inputs to different chains based on certain conditions. It's useful when you need to handle different types of inputs or tasks within the same workflow.

GraphChain: This chain allows you to create a graph of chains, where each node in the graph represents a chain. This is useful for more complex workflows that require branching and merging of different chains.

MemoryChain: This chain is designed to maintain a memory of previous interactions. It's particularly useful for applications that require context retention, such as chatbots or conversational agents.

AgentChain: This chain is used to create agents that can perform specific tasks. Agents can use tools, access APIs, and interact with other systems to complete their tasks.

Retrieval-Augmented Generation (RAG) Chain: This chain combines retrieval-based methods with generation-based methods to improve the quality and relevance of the generated content. It's useful for tasks that require accessing external knowledge or databases

In [ ]:
def apprendModeles(X, y, labelDataApprises):
    
    
    print(str(datetime.now())+" - __start__")
    with Timer() as timer:
        vectRun = vectorizer.fit(X)
    print(str(datetime.now())+' - temps de traitement du fitting : '+str(timer.interval))
    with Timer() as timer:
        X_RUN = vectRun.transform(X)
    print(str(datetime.now())+' - temps de traitement du transform : '+str(timer.interval))
    dump(vectRun, 'vectRun.modele')

In [ ]:
X = [
    "I love programming in Python.",
    "Machine learning is fascinating.",
    "Natural language processing is a complex field.",
    "Deep learning models are powerful.",
    "Data science involves statistics and programming."
]
y = [0, 1, 1, 1, 0]